# GPU-Accelerated Curve Fitting with Theseus
Theseus is used to provide second-order nonlinear optimizers for efficient and high-throughput curve fitting.

In [1]:
import numpy as np
import h5py
import glob
from theseusqfls import genplanck, visualization
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

ImportError: dlopen(/Users/cbchosy/opt/anaconda3/envs/theseus/lib/python3.10/site-packages/sksparse/cholmod.cpython-310-darwin.so, 0x0002): Library not loaded: @rpath/libblas.3.dylib
  Referenced from: <F0E743CE-ABC7-3A11-843A-3D271FDCFDA6> /Users/cbchosy/opt/anaconda3/envs/theseus/lib/libcholmod.3.0.14.dylib
  Reason: tried: '/Users/cbchosy/opt/anaconda3/envs/theseus/lib/libblas.3.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/cbchosy/opt/anaconda3/envs/theseus/lib/libblas.3.dylib' (no such file), '/Users/cbchosy/opt/anaconda3/envs/theseus/lib/libblas.3.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/cbchosy/opt/anaconda3/envs/theseus/lib/libblas.3.dylib' (no such file), '/Users/cbchosy/opt/anaconda3/envs/theseus/lib/libblas.3.dylib' (no such file), '/Users/cbchosy/opt/anaconda3/envs/theseus/lib/libblas.3.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/cbchosy/opt/anaconda3/envs/theseus/lib/libblas.3.dylib' (no such file), '/Us

In [ ]:
def calculate_guesses(energy, pl):
    init_guesses = [1.2, 0.025, 1.5, 1.7]
    points = [256, 512, 768]
    guesses = np.zeros((4, len(points), len(points)))
    for i in range(len(points)):
        for j in range(len(points)):
            guesses[:, i, j], __ = curve_fit(genplanck.gen_planck, energy, pl[:, i, j].squeeze(), p0=init_guesses)
    guesses = list(np.mean(guesses, axis=(1, 2)))
    return guesses

In [1]:
directory = r'/Volumes/My Passport/macro-nano-degradation/hyperspectral/round2/fresh/processed'
filenames = glob.glob(directory + '/*_calibrated.h5')

In [ ]:
for filename in filenames:
    energy, pl = genplanck.import_hyperspectral(filename)
    energy_crop, pl_crop = genplanck.crop_data(energy, pl)
    guesses = calculate_guesses(energy_crop, pl_crop)
    result, info = genplanck.fit_qfls(energy_crop, pl_crop, guesses=guesses, batch_size=2048, max_iterations=10, step_size=0.75, verbose=False)
    # Plot result
    fig, ax = plt.subplots()
    visualization.imshow(ax, result['qfls'], vmin=np.nanpercentile(result['qfls'], 1), vmax=np.nanpercentile(result['qfls'], 99), cbar_label='QFLS (eV)')
    plt.show()